<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # keep track of iterations
    i = 0
    # conditional stop variable
    stop = False
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    while(not stop):
        # increase iteration count
        i += 1
        # check that max iterations hasn't been passed
        # if it has been passed, return False to indicate failure
        # also return the max number of iterations and conds
        if i > it:
            return None, None, it, conds
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i, conds


**RQI Algorithm with Additive Preconditioning**

In [4]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # keep track of iterations
    i = 0
    # conditional stop variable
    stop = False
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    while(not stop):
        # increase iteration count
        i += 1
        # check that max iterations hasn't been passed
        # if it has been passed, return False to indicate failure
        # also return the max number of iterations and conds
        if i > it:
            return None, None, it, conds, conds_C, conds_desired
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x) <= ep*np.linalg.norm(M):
            return mu, y, i, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [5]:
# Setup

# create a random matrix M
M = np.random.rand(10, 10)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [6]:
# Print condition number of M
svd_cond(M)

169.82119455392856

In [7]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (4.919979238757471+0j)
Eigenvector: [0.26570035+0.j 0.28681472+0.j 0.32432816+0.j 0.35162262+0.j
 0.30510703+0.j 0.35879549+0.j 0.24291795+0.j 0.31067208+0.j
 0.33417086+0.j 0.35957262+0.j]
Iterations: 3
Condition Numbers: [2.32634457e+01 1.70866091e+03 6.34422175e+07]


In [8]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (4.919985034182345+0j)
Eigenvector: [0.26570937+0.j 0.28680918+0.j 0.32430547+0.j 0.3516292 +0.j
 0.30509134+0.j 0.35879016+0.j 0.24290626+0.j 0.31068717+0.j
 0.33417957+0.j 0.35958982+0.j]
Iterations: 2
Condition Numbers for (mu)I-M: [ 23.26344565 779.87249717]
Condition Numbers for C: [4.00125603 4.05553781]
Desired Condition Numbers: [1.56740826 1.53193781]


In [9]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (4.9199792387574615+0j)
Eigenvector: [0.26570035+0.j 0.28681472+0.j 0.32432816+0.j 0.35162262+0.j
 0.30510703+0.j 0.35879549+0.j 0.24291795+0.j 0.31067208+0.j
 0.33417086+0.j 0.35957262+0.j]
Iterations: 4
Condition Numbers: [2.32634457e+01 1.90141743e+02 3.03704435e+04 6.33599656e+08]


In [10]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (4.919979238757587+0j)
Eigenvector: [0.26570035+0.j 0.28681472+0.j 0.32432816+0.j 0.35162262+0.j
 0.30510703+0.j 0.35879549+0.j 0.24291795+0.j 0.31067208+0.j
 0.33417086+0.j 0.35957262+0.j]
Iterations: 3
Condition Numbers: [2.32634457e+01 2.42666682e+03 3.67318636e+06]
Condition Numbers for C: [4.13004543 4.22641526 4.21247506]
Desired Condition Numbers: [1.56740826 1.5305331  1.53087426]


In [11]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.32634457e+01 2.39046185e+02 4.75110508e+04 1.46317269e+09
 8.46792698e+15 3.99631359e+16 5.03300939e+15 3.55676235e+15
 2.56510471e+15 3.99631359e+16 8.46792698e+15 2.56510471e+15
 3.99631359e+16 8.46792698e+15 2.56510471e+15 3.99631359e+16
 8.46792698e+15 2.56510471e+15 3.99631359e+16 8.46792698e+15
 2.56510471e+15 3.99631359e+16 8.46792698e+15 2.56510471e+15
 3.99631359e+16 8.46792698e+15 2.56510471e+15 3.99631359e+16
 8.46792698e+15 2.56510471e+15 3.99631359e+16 8.46792698e+15
 2.56510471e+15 3.99631359e+16 8.46792698e+15 2.56510471e+15
 3.99631359e+16 8.46792698e+15 2.56510471e+15 3.99631359e+16
 8.46792698e+15 2.56510471e+15 3.99631359e+16 8.46792698e+15
 2.56510471e+15 3.99631359e+16 8.46792698e+15 2.56510471e+15
 3.99631359e+16 8.46792698e+15 2.56510471e+15 3.99631359e+16
 8.46792698e+15 2.56510471e+15 3.99631359e+16 8.46792698e+15
 2.56510471e+15 3.99631359e+16 8.46792698e+15 2.56510471e+15
 3.99631359e+16

In [12]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (4.919979240249256+0j)
Eigenvector: [0.26570035+0.j 0.28681472+0.j 0.32432816+0.j 0.35162262+0.j
 0.30510703+0.j 0.35879549+0.j 0.24291795+0.j 0.31067208+0.j
 0.33417086+0.j 0.35957262+0.j]
Iterations: 3
Condition Numbers: [2.32634457e+01 3.15957412e+02 7.84103127e+04]
Condition Numbers for C: [4.40975096 4.19142788 4.19143801]
Desired Condition Numbers: [1.56740826 1.53350195 1.53086392]


In [13]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(4.9199792387574615+0j) [-0.26570035+0.j -0.28681472+0.j -0.32432816+0.j -0.35162262+0.j
 -0.30510703+0.j -0.35879549+0.j -0.24291795+0.j -0.31067208+0.j
 -0.33417086+0.j -0.35957262+0.j]
